# CatBoost OOF + SoftVoting Ensemble (GPU 지원) — Logloss 학습 + Global FE 캐싱 + 10%/50%/Full 모드

실행 순서: **[0] 설정/상수 → [1] 유틸 → [2] FE 캐시 → [3] OOF 학습/seed 예측 → [4] SoftVoting/평가**


## [0] 설정 / 상수 (먼저 실행)

In [ ]:

import os

class CFG:
    DATA_PATH = "/home/konkukstat/python3/workspace/data"
    CACHE_SUBDIR = "_cache_fe"
    OUTPUT_DIR = "./outputs"
    
    TRAIN_FULL_FILE = "train.parquet"
    TRAIN_10_FILE = "train_sample_10pct.parquet"
    TRAIN_50_FILE = "train_sampled_50pct.parquet"
    TEST_FILE = "test.parquet"

    TRAIN_MODE = "FULL"
    SAMPLE_SEED = 42

    N_SPLITS = 5
    SEEDS = [42, 106, 1031]
    SEED_WEIGHTS = None   # ✅ 추가 (None이면 균등 평균)
    TEST_LABEL_FILE = None
    
    OUTPUT_PREFIX = "CB"

    # ---- CatBoost Params ----
    CB_ITERATIONS = 2000
    CB_LEARNING_RATE = 0.05
    CB_DEPTH = 7
    CB_L2 = 20
    CB_RANDOM_STRENGTH = 1.0
    CB_SUBSAMPLE = 0.8

    USE_GPU = True
    GPU_DEVICES = "0-1"
    
    # =========================
    # Refit Settings
    # =========================
    REFIT_ITERS_MIN = 300
    REFIT_ITERS_MAX = 5000
    REFIT_RECOMPUTE_TE = True   # full-train 기준 TE 다시 계산 여부

print("✅ CFG:", CFG.TRAIN_MODE, "| folds:", CFG.N_SPLITS, "| seeds:", CFG.SEEDS, "| GPU:", CFG.USE_GPU, CFG.GPU_DEVICES)


✅ CFG: FULL | folds: 5 | seeds: [42, 106, 1031] | GPU: True 0-1


## [1] 라이브러리 / 유틸 함수 (먼저 실행)

In [11]:

import gc, random, time
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, log_loss

import catboost as cb

EPS = 1e-15

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

def toss_score(y_true, y_pred, eps=1e-15):
    '''
    Temporary Toss definition.
    Replace with your exact competition/project Toss if needed.
    Current: 0.5*AP + 0.5*(1/(1+Logloss))
    '''
    y_pred = np.clip(np.asarray(y_pred, dtype=float), eps, 1-eps)
    ap = float(average_precision_score(y_true, y_pred))
    ll = float(log_loss(y_true, y_pred))
    return 0.5*ap + 0.5*(1.0/(1.0+ll))

def add_cat_crosses(train_df: pd.DataFrame, test_df: pd.DataFrame, pairs):
    for a, b in pairs:
        if a in train_df.columns and b in train_df.columns and a in test_df.columns and b in test_df.columns:
            new = f"{a}__x__{b}"
            train_df[new] = train_df[a].astype(str) + "_" + train_df[b].astype(str)
            test_df[new]  = test_df[a].astype(str)  + "_" + test_df[b].astype(str)
    return train_df, test_df

def add_count_features(train_df: pd.DataFrame, test_df: pd.DataFrame, cols, use_test_in_counts=True):
    for c in cols:
        if c not in train_df.columns or c not in test_df.columns:
            continue
        all_series = pd.concat([train_df[c], test_df[c]]) if use_test_in_counts else train_df[c]
        vc = all_series.astype(str).value_counts(dropna=False)
        new = f"{c}__cnt"
        train_df[new] = train_df[c].astype(str).map(vc).astype(np.float32)
        test_df[new]  = test_df[c].astype(str).map(vc).astype(np.float32)
    return train_df, test_df

def add_seq_features(df: pd.DataFrame, mode: str, fill_stats=None):
    # df 전체 copy 하지 말고, 새 컬럼만 추가(원본을 바꾸고 싶지 않다면 df = df.copy(deep=False) 정도만)
    out = df  # in-place로 붙이는 버전 (가장 메모리 절약)
    # out = df.copy(deep=False)  # 원본 보호가 필요하면 이 정도만

    if "seq" in out.columns:
        s = out["seq"]

        # 결측만 처리: astype(str) 전체 변환은 비쌈.
        # 문자열이 아닌 값이 섞여있을 수 있어 안전하게 문자열화는 하되, NA만 먼저 채우고 변환
        s2 = s.fillna("").astype("string")  # pandas StringDtype (object보다 보통 안정적)

        # split 대신 partition / rpartition (리스트 생성 방지)
        # 첫 토큰: 'a,b,c' -> 'a'
        first = s2.str.partition(",")[0]
        # 마지막 토큰: 'a,b,c' -> 'c'
        last  = s2.str.rpartition(",")[2]

        seq_first = pd.to_numeric(first, errors="coerce").astype("float32")
        seq_last  = pd.to_numeric(last,  errors="coerce").astype("float32")
    else:
        seq_first = pd.Series(np.nan, index=out.index, dtype="float32")
        seq_last  = pd.Series(np.nan, index=out.index, dtype="float32")

    if fill_stats is None:
        m1 = float(seq_first.mean(skipna=True))
        m2 = float(seq_last.mean(skipna=True))
        if not np.isfinite(m1): m1 = 0.0
        if not np.isfinite(m2): m2 = 0.0
        fill_stats = {"seq_first_num_mean": m1, "seq_last_num_mean": m2}

    out["seq_first_num"] = seq_first.fillna(fill_stats["seq_first_num_mean"]).astype("float32")
    out["seq_last_num"]  = seq_last.fillna(fill_stats["seq_last_num_mean"]).astype("float32")

    return out, fill_stats

def add_oof_target_encoding(train_df: pd.DataFrame, test_df: pd.DataFrame, y, cols, folds, smoothing=20.0, noise=0.0, seed=42):
    rng = np.random.default_rng(seed)
    y = np.asarray(y).astype(float)
    global_mean = float(np.mean(y))

    if "clicked" not in train_df.columns:
        raise KeyError("train_df must include 'clicked' for TE.")

    for c in cols:
        if c not in train_df.columns or c not in test_df.columns:
            continue

        te_col = f"{c}__te"
        train_te = np.zeros(len(train_df), dtype=np.float32)

        for tr_idx, va_idx in folds:
            tr = train_df.iloc[tr_idx]
            va = train_df.iloc[va_idx]
            stats = tr.groupby(c)["clicked"].agg(["mean", "count"])
            means = stats["mean"]
            cnts  = stats["count"]
            smooth = (means*cnts + global_mean*smoothing) / (cnts + smoothing)

            mapped = va[c].map(smooth).fillna(global_mean).astype(np.float32).values
            if noise > 0:
                mapped = mapped * (1.0 + rng.normal(0, noise, size=len(mapped))).astype(np.float32)
            train_te[va_idx] = mapped

        stats_full = train_df.groupby(c)["clicked"].agg(["mean", "count"])
        means = stats_full["mean"]
        cnts  = stats_full["count"]
        smooth_full = (means*cnts + global_mean*smoothing) / (cnts + smoothing)
        test_te = test_df[c].map(smooth_full).fillna(global_mean).astype(np.float32).values

        train_df[te_col] = train_te
        test_df[te_col]  = test_te

    return train_df, test_df

def make_groupby_stats(source_df: pd.DataFrame, key: str, agg_dict: dict):
    usable = {k: v for k, v in agg_dict.items() if k in source_df.columns}
    if key not in source_df.columns or len(usable) == 0:
        return None
    stats = source_df.groupby(key).agg(usable)
    stats.columns = [f"{key}_{a}_{b}" for (a, b) in stats.columns]
    stats = stats.reset_index()
    return stats

def merge_groupby_stats(df: pd.DataFrame, stats: pd.DataFrame, key: str, fill_from_df: pd.DataFrame):
    if stats is None:
        return df
    df = df.merge(stats, on=key, how="left")
    new_cols = [c for c in stats.columns if c != key]
    for c in new_cols:
        m = float(np.nanmean(fill_from_df[c].values)) if c in fill_from_df.columns else 0.0
        if not np.isfinite(m):
            m = 0.0
        df[c] = df[c].fillna(m)
    return df

def print_gpu_info():
    try:
        import subprocess
        out = subprocess.check_output("nvidia-smi -L", shell=True).decode()
        lines = [ln for ln in out.splitlines() if ln.strip()]
        print("✅ GPUs detected:", len(lines))
        print(out)
    except Exception as e:
        print("⚠️ nvidia-smi not available:", e)

print("✅ utilities loaded")


✅ utilities loaded


## [1.1] 피처 설정

In [12]:

DROP_COLS_RAW = ["clicked", "ID", "seq"]

BASE_CAT_COLS = ["gender", "age_group", "inventory_id", "day_of_week", "hour", "seq_first_num", "seq_last_num"]

CROSS_PAIRS = [
    ("inventory_id", "hour"),
    ("inventory_id", "day_of_week"),
    ("inventory_id", "age_group"),
    ("gender", "age_group"),
]

TE_COLS = ["inventory_id", "inv_hour", "inv_dow"]

AGGS_TO_CREATE = {
    "history_a_1": ["mean", "std"], "history_a_2": ["mean", "std"],
    "history_a_3": ["mean", "std"], "history_a_6": ["mean", "std"],
    "feat_e_2": ["mean", "std"], "feat_e_3": ["mean", "std"],
    "feat_c_8": ["mean", "std"], "feat_e_9": ["mean", "std"],
    "feat_d_4": ["mean", "std"],
    "l_feat_1": ["mean", "std"], "l_feat_2": ["mean", "std"],
    "l_feat_5": ["mean", "std"], "l_feat_7": ["mean", "std"],
    "l_feat_10": ["mean", "std"], "l_feat_15": ["mean", "std"],
}

print("✅ feature specs loaded")


✅ feature specs loaded


## [2] Global FE 캐시 생성/로드 (TRAIN_MODE당 1회)

In [13]:

CACHE_DIR = os.path.join(CFG.DATA_PATH, CFG.CACHE_SUBDIR)
os.makedirs(CACHE_DIR, exist_ok=True)

mode = str(CFG.TRAIN_MODE).lower().strip()
TRAIN_FE_CACHE = os.path.join(CACHE_DIR, f"train_global_fe_{mode}.parquet")
TEST_FE_CACHE  = os.path.join(CACHE_DIR, f"test_global_fe_{mode}.parquet")

def get_train_file_by_mode(mode):
    m = str(mode).lower().strip()
    if m == "full":
        return CFG.TRAIN_FULL_FILE
    if m == "10":
        return CFG.TRAIN_10_FILE
    if m == "50":
        return CFG.TRAIN_50_FILE
    raise ValueError("CFG.TRAIN_MODE must be one of: '10', '50', 'full'")

def load_raw_train_test():
    train_file = get_train_file_by_mode(CFG.TRAIN_MODE)
    train_path = os.path.join(CFG.DATA_PATH, train_file)
    full_path  = os.path.join(CFG.DATA_PATH, CFG.TRAIN_FULL_FILE)

    if os.path.exists(train_path):
        print(f"📂 Loading train: {train_path}")
        train_df = pd.read_parquet(train_path).reset_index(drop=True)
    else:
        if not os.path.exists(full_path):
            raise FileNotFoundError(f"train file not found: {train_path} and full not found: {full_path}")
        frac = 0.10 if mode == "10" else (0.50 if mode == "50" else None)
        print(f"⚠️ {train_file} not found -> sample frac={frac} from {full_path}")
        train_df = pd.read_parquet(full_path).reset_index(drop=True)
        if frac is not None:
            train_df = train_df.sample(frac=frac, random_state=CFG.SAMPLE_SEED).reset_index(drop=True)

    train_df["row_id"] = np.arange(len(train_df))

    test_path = os.path.join(CFG.DATA_PATH, CFG.TEST_FILE)
    if not os.path.exists(test_path):
        raise FileNotFoundError(f"test file not found: {test_path}")
    print(f"📂 Loading test : {test_path}")
    test_df = pd.read_parquet(test_path).reset_index(drop=True)

    print(f"✅ Raw Train: {train_df.shape} | Raw Test: {test_df.shape}")
    return train_df, test_df

def build_global_fe_and_cache():
    train_df, test_df = load_raw_train_test()
    if "clicked" not in train_df.columns:
        raise KeyError("'clicked' not found in train")

    print(f"📌 click-rate = {float(train_df['clicked'].mean()):.6f}")

    print("🛠️ add_seq_features ...")
    train_df, seq_fill = add_seq_features(train_df, "train", fill_stats=None)
    test_df, _ = add_seq_features(test_df, "test", fill_stats=seq_fill)

    # 더 이상 seq 안 쓰면 드랍
    for df in (train_df, test_df):
        if "seq" in df.columns:
            df.drop(columns=["seq"], inplace=True)
    
    print("🛠️ add_cat_crosses ...")
    train_df, test_df = add_cat_crosses(train_df, test_df, CROSS_PAIRS)

    print("🛠️ add_count_features ...")
    count_cols = [c for c in BASE_CAT_COLS if c in train_df.columns and c in test_df.columns]
    if len(count_cols) > 0:
        train_df, test_df = add_count_features(train_df, test_df, count_cols, use_test_in_counts=True)

    train_df.to_parquet(TRAIN_FE_CACHE, index=False)
    test_df.to_parquet(TEST_FE_CACHE, index=False)
    print("💾 Saved:", TRAIN_FE_CACHE)
    print("💾 Saved:", TEST_FE_CACHE)

    del train_df, test_df
    gc.collect()

if not (os.path.exists(TRAIN_FE_CACHE) and os.path.exists(TEST_FE_CACHE)):
    print("🧊 FE cache not found -> building once...")
    build_global_fe_and_cache()
else:
    print("🧊 FE cache exists -> skip build")

print("🧊 Loading FE cache ...")
train_base = pd.read_parquet(TRAIN_FE_CACHE).reset_index(drop=True)
test_base  = pd.read_parquet(TEST_FE_CACHE).reset_index(drop=True)
print(f"✅ Cached Train: {train_base.shape} | Cached Test: {test_base.shape}")


🧊 FE cache not found -> building once...
📂 Loading train: /home/konkukstat/python3/workspace/data/train.parquet
📂 Loading test : /home/konkukstat/python3/workspace/data/test.parquet
✅ Raw Train: (10704179, 120) | Raw Test: (1527298, 119)
📌 click-rate = 0.019075
🛠️ add_seq_features ...
🛠️ add_cat_crosses ...
🛠️ add_count_features ...
💾 Saved: /home/konkukstat/python3/workspace/data/_cache_fe/train_global_fe_full.parquet
💾 Saved: /home/konkukstat/python3/workspace/data/_cache_fe/test_global_fe_full.parquet
🧊 Loading FE cache ...
✅ Cached Train: (10704179, 132) | Cached Test: (1527298, 131)


## [3] Main OOF Training Loop (seed 3개 + GPU 지원) — seed별 pred 저장

In [14]:

prediction_files = []
oof_files = []
seed_test_preds = {}
seed_oof_preds  = {}
seed_best_iters = {}

if CFG.USE_GPU:
    print_gpu_info()

for seed in CFG.SEEDS:
    print(f"\n{'='*40}\n🚀 Processing Seed: {seed} | mode={CFG.TRAIN_MODE}\n{'='*40}")
    seed_everything(seed)

    train_df = train_base.copy()
    test_df  = test_base.copy()

    y_all = train_df["clicked"].values.astype(int)
    print(f"📌 click-rate = {float(y_all.mean()):.6f}")

    skf = StratifiedKFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=seed)
    folds_list = list(skf.split(train_df, y_all))

    train_df, test_df = add_oof_target_encoding(
        train_df, test_df, y_all,
        cols=TE_COLS,
        folds=folds_list,
        smoothing=20.0,
        noise=0.0,
        seed=seed
    )

    DROP_COLS = [c for c in DROP_COLS_RAW if c in train_df.columns]
    feature_cols_base = [c for c in train_df.columns if c not in DROP_COLS + ["row_id"]]

    auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()
    cat_cols_base = sorted(list(set([c for c in BASE_CAT_COLS if c in feature_cols_base] + auto_cat_cols)))
    cat_cols_base = [c for c in cat_cols_base if c in test_df.columns]

    print(f"✅ base features={len(feature_cols_base)} | cat_cols={len(cat_cols_base)}")

    oof_pred = np.zeros(len(train_df), dtype=float)
    test_pred_folds = []
    best_iters = []

    cb_params = dict(
        loss_function="Logloss",
        eval_metric="Logloss",
        iterations=CFG.CB_ITERATIONS,
        learning_rate=CFG.CB_LEARNING_RATE,
        depth=CFG.CB_DEPTH,
        l2_leaf_reg=CFG.CB_L2,
        random_strength=CFG.CB_RANDOM_STRENGTH,
        auto_class_weights="Balanced",
        bootstrap_type="Bernoulli",
        subsample=CFG.CB_SUBSAMPLE,
        random_state=seed,
        verbose=False,
        allow_writing_files=False,
    )

    if CFG.USE_GPU:
        cb_params.update(dict(task_type="GPU", devices=CFG.GPU_DEVICES))
        # GPU에서는 rsm 미지원 -> 넣지 않음
    else:
        cb_params.update(dict(task_type="CPU", rsm=CFG.CB_RSM_CPU))

    for fold, (tr_idx, va_idx) in enumerate(folds_list, 1):
        print(f"  🔁 Fold {fold}/{CFG.N_SPLITS}")

        X_tr = train_df.iloc[tr_idx].copy()
        y_tr = y_all[tr_idx]
        X_va = train_df.iloc[va_idx].copy()
        y_va = y_all[va_idx]
        X_te = test_df.copy()

        # fold-safe groupby stats
        group_key = "inventory_id"
        groupby_cols = []
        if group_key in X_tr.columns:
            agg_dict = {k:v for k,v in AGGS_TO_CREATE.items() if k in X_tr.columns}
            if len(agg_dict) > 0:
                stats = make_groupby_stats(X_tr, group_key, agg_dict)
                X_tr = merge_groupby_stats(X_tr, stats, group_key, fill_from_df=X_tr)
                X_va = merge_groupby_stats(X_va, stats, group_key, fill_from_df=X_tr)
                X_te = merge_groupby_stats(X_te, stats, group_key, fill_from_df=X_tr)
                groupby_cols = [c for c in stats.columns if c != group_key]

        feature_cols_fold = feature_cols_base + groupby_cols
        cat_cols_fold = [c for c in cat_cols_base if c in feature_cols_fold]

        # 타입/결측 안정화
        for col in feature_cols_fold:
            if col in cat_cols_fold:
                X_tr[col] = X_tr[col].astype(str).fillna("unknown")
                X_va[col] = X_va[col].astype(str).fillna("unknown")
                X_te[col] = X_te[col].astype(str).fillna("unknown")
            else:
                X_tr[col] = pd.to_numeric(X_tr[col], errors="coerce")
                X_va[col] = pd.to_numeric(X_va[col], errors="coerce")
                X_te[col] = pd.to_numeric(X_te[col], errors="coerce")
                m = X_tr[col].mean()
                fv = float(m) if np.isfinite(m) else 0.0
                X_tr[col] = X_tr[col].fillna(fv)
                X_va[col] = X_va[col].fillna(fv)
                X_te[col] = X_te[col].fillna(fv)

        model = cb.CatBoostClassifier(**cb_params, cat_features=cat_cols_fold)
        model.fit(
            X_tr[feature_cols_fold], y_tr,
            eval_set=(X_va[feature_cols_fold], y_va),
            early_stopping_rounds=100,
            use_best_model=True,
            verbose=False
        )

        best_it = model.get_best_iteration()
        if best_it is None:
            best_it = cb_params.get("iterations", 2000)
        best_iters.append(int(best_it))

        va_pred = model.predict_proba(X_va[feature_cols_fold])[:, 1]
        te_pred = model.predict_proba(X_te[feature_cols_fold])[:, 1]

        oof_pred[va_idx] = va_pred
        test_pred_folds.append(te_pred)

        ll = float(log_loss(y_va, np.clip(va_pred, EPS, 1-EPS)))
        ap = float(average_precision_score(y_va, np.clip(va_pred, EPS, 1-EPS)))
        ts = float(toss_score(y_va, va_pred))
        print(f"     LogLoss={ll:.6f} | AP={ap:.6f} | Toss={ts:.6f}")

        del model, X_tr, X_va, X_te
        gc.collect()

    # OOF Summary
    oof_clip = np.clip(oof_pred, EPS, 1-EPS)
    val_ll = float(log_loss(y_all, oof_clip))
    val_ap = float(average_precision_score(y_all, oof_clip))
    val_ts = float(toss_score(y_all, oof_clip))
    print(f"✅ OOF Summary: LogLoss={val_ll:.6f} | AP={val_ap:.6f} | Toss={val_ts:.6f}")

    # fold 평균 test pred
    test_pred = np.mean(np.vstack(test_pred_folds), axis=0)

    # save
    os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)
    pred_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_{mode}_seed{seed}_pred.csv")
    pd.DataFrame({"ID": test_df["ID"].values, "clicked": test_pred}).to_csv(pred_file, index=False)
    prediction_files.append(pred_file)
    print("💾 Pred saved:", pred_file)

    oof_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_{mode}_seed{seed}_oof.csv")
    pd.DataFrame({"row_id": train_df["row_id"].values, "y_true": y_all, "oof_pred": oof_pred}).to_csv(oof_file, index=False)
    oof_files.append(oof_file)
    print("💾 OOF saved:", oof_file)

    seed_test_preds[seed] = test_pred
    seed_oof_preds[seed]  = oof_pred
    seed_best_iters[seed] = best_iters

    del train_df, test_df
    gc.collect()

print("✅ Done. pred files:", prediction_files)


✅ GPUs detected: 2
GPU 0: NVIDIA GeForce RTX 4090 (UUID: GPU-ec4c30bc-a6f3-154d-6fc1-05d9a9956795)
GPU 1: NVIDIA GeForce RTX 4090 (UUID: GPU-e3160e3d-1847-13ee-9755-7b93df3dc8cf)


🚀 Processing Seed: 42 | mode=FULL
📌 click-rate = 0.019075


/tmp/ipykernel_449002/4172781264.py:35: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()


✅ base features=131 | cat_cols=11
  🔁 Fold 1/5
     LogLoss=0.574741 | AP=0.083069 | Toss=0.359047
  🔁 Fold 2/5
     LogLoss=0.573751 | AP=0.082250 | Toss=0.358838
  🔁 Fold 3/5
     LogLoss=0.577117 | AP=0.082154 | Toss=0.358111
  🔁 Fold 4/5
     LogLoss=0.572230 | AP=0.083351 | Toss=0.359695
  🔁 Fold 5/5
     LogLoss=0.572642 | AP=0.083381 | Toss=0.359627
✅ OOF Summary: LogLoss=0.574096 | AP=0.082809 | Toss=0.359047
💾 Pred saved: ./outputs/CB_full_seed42_pred.csv
💾 OOF saved: ./outputs/CB_full_seed42_oof.csv

🚀 Processing Seed: 106 | mode=FULL
📌 click-rate = 0.019075


/tmp/ipykernel_449002/4172781264.py:35: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()


✅ base features=131 | cat_cols=11
  🔁 Fold 1/5
     LogLoss=0.574018 | AP=0.083259 | Toss=0.359288
  🔁 Fold 2/5
     LogLoss=0.573211 | AP=0.083153 | Toss=0.359398
  🔁 Fold 3/5
     LogLoss=0.570663 | AP=0.083137 | Toss=0.359906
  🔁 Fold 4/5
     LogLoss=0.573184 | AP=0.081983 | Toss=0.358818
  🔁 Fold 5/5
     LogLoss=0.574103 | AP=0.083157 | Toss=0.359220
✅ OOF Summary: LogLoss=0.573036 | AP=0.082895 | Toss=0.359304
💾 Pred saved: ./outputs/CB_full_seed106_pred.csv
💾 OOF saved: ./outputs/CB_full_seed106_oof.csv

🚀 Processing Seed: 1031 | mode=FULL
📌 click-rate = 0.019075


/tmp/ipykernel_449002/4172781264.py:35: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()


✅ base features=131 | cat_cols=11
  🔁 Fold 1/5
     LogLoss=0.573489 | AP=0.082680 | Toss=0.359105
  🔁 Fold 2/5
     LogLoss=0.573164 | AP=0.082047 | Toss=0.358854
  🔁 Fold 3/5
     LogLoss=0.574955 | AP=0.082705 | Toss=0.358822
  🔁 Fold 4/5
     LogLoss=0.567496 | AP=0.082995 | Toss=0.360478
  🔁 Fold 5/5
     LogLoss=0.574900 | AP=0.083733 | Toss=0.359347
✅ OOF Summary: LogLoss=0.572801 | AP=0.082799 | Toss=0.359304
💾 Pred saved: ./outputs/CB_full_seed1031_pred.csv
💾 OOF saved: ./outputs/CB_full_seed1031_oof.csv
✅ Done. pred files: ['./outputs/CB_full_seed42_pred.csv', './outputs/CB_full_seed106_pred.csv', './outputs/CB_full_seed1031_pred.csv']


## [3.1] Full-train Refit (seed별 best_iter 평균) + seed별 test 예측 저장

- [3]에서 각 fold의 `best_iteration`을 모아 평균으로 `refit_iters`를 정하고,
  같은 피처 파이프라인으로 **전체 train**에 다시 학습한 뒤 test를 예측합니다.
- fold별 test 예측 평균보다 **일반적으로 더 강한 최종 모델**이 됩니다.
- (선택) refit 단계에서는 Target Encoding을 **전체 train 기준으로 재계산**해서 적용할 수 있습니다.


In [15]:

# --- Full-train refit per seed (uses best_iters from [3]) ---

seed_test_preds_refit = {}

def compute_full_target_encoding(train_df, test_df, y, cols, smoothing=20.0):
    """전체 train 기준 TE를 계산해서 train/test에 동일 적용 (refit 단계용)."""
    prior = float(np.mean(y))
    out_tr = train_df.copy()
    out_te = test_df.copy()
    for col in cols:
        if col not in out_tr.columns or col not in out_te.columns:
            continue
        grp = out_tr.groupby(col)["clicked"].agg(["count","mean"]).reset_index()
        grp["te"] = (grp["count"]*grp["mean"] + smoothing*prior) / (grp["count"] + smoothing)
        mapping = dict(zip(grp[col].values, grp["te"].values))
        out_tr[f"te_{col}"] = out_tr[col].map(mapping).fillna(prior).astype(float)
        out_te[f"te_{col}"] = out_te[col].map(mapping).fillna(prior).astype(float)
    return out_tr, out_te

if "seed_best_iters" not in globals():
    raise RuntimeError("Run [3] first. seed_best_iters not found.")

for seed in CFG.SEEDS:
    if seed not in seed_best_iters:
        print(f"⚠️ seed {seed}: best_iters not found -> skip refit")
        continue

    best_iters = seed_best_iters[seed]
    refit_iters = int(np.clip(np.mean(best_iters), CFG.REFIT_ITERS_MIN, CFG.REFIT_ITERS_MAX))
    print(f"\n🧠 Refit seed={seed} | refit_iters={refit_iters} | mode={CFG.TRAIN_MODE}")

    train_df = train_base.copy()
    test_df  = test_base.copy()
    y_all = train_df["clicked"].values.astype(int)

    # (선택) 최종 refit에서는 TE를 full-train 기준으로 재계산
    if CFG.REFIT_RECOMPUTE_TE:
        train_df, test_df = compute_full_target_encoding(train_df, test_df, y_all, TE_COLS, smoothing=20.0)

    DROP_COLS = [c for c in DROP_COLS_RAW if c in train_df.columns]
    feature_cols_base = [c for c in train_df.columns if c not in DROP_COLS + ["row_id"]]
    auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()
    cat_cols_base = sorted(list(set([c for c in BASE_CAT_COLS if c in feature_cols_base] + auto_cat_cols)))
    cat_cols_base = [c for c in cat_cols_base if c in test_df.columns]

    # full 기준 groupby stats
    group_key = "inventory_id"
    groupby_cols = []
    if group_key in train_df.columns:
        agg_dict = {k:v for k,v in AGGS_TO_CREATE.items() if k in train_df.columns}
        if len(agg_dict) > 0:
            stats = make_groupby_stats(train_df, group_key, agg_dict)
            train_df = merge_groupby_stats(train_df, stats, group_key, fill_from_df=train_df)
            test_df  = merge_groupby_stats(test_df,  stats, group_key, fill_from_df=train_df)
            groupby_cols = [c for c in stats.columns if c != group_key]

    feature_cols = feature_cols_base + groupby_cols
    cat_cols = [c for c in cat_cols_base if c in feature_cols]

    # 타입/결측 안정화
    for col in feature_cols:
        if col in cat_cols:
            train_df[col] = train_df[col].astype(str).fillna("unknown")
            test_df[col]  = test_df[col].astype(str).fillna("unknown")
        else:
            train_df[col] = pd.to_numeric(train_df[col], errors="coerce")
            test_df[col]  = pd.to_numeric(test_df[col], errors="coerce")
            m = train_df[col].mean()
            fv = float(m) if np.isfinite(m) else 0.0
            train_df[col] = train_df[col].fillna(fv)
            test_df[col]  = test_df[col].fillna(fv)

    cb_params = dict(
        loss_function="Logloss",
        eval_metric="Logloss",
        iterations=refit_iters,
        learning_rate=CFG.CB_LEARNING_RATE,
        depth=CFG.CB_DEPTH,
        l2_leaf_reg=CFG.CB_L2,
        random_strength=CFG.CB_RANDOM_STRENGTH,
        auto_class_weights="Balanced",
        bootstrap_type="Bernoulli",
        subsample=CFG.CB_SUBSAMPLE,
        random_state=seed,
        verbose=False,
        allow_writing_files=False,
    )
    if CFG.USE_GPU:
        cb_params.update(dict(task_type="GPU", devices=CFG.GPU_DEVICES))
    else:
        cb_params.update(dict(task_type="CPU", rsm=CFG.CB_RSM_CPU))

    model = cb.CatBoostClassifier(**cb_params, cat_features=cat_cols)
    model.fit(train_df[feature_cols], y_all, verbose=False)
    pred = model.predict_proba(test_df[feature_cols])[:, 1].astype(float)

    os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)
    refit_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_{mode}_seed{seed}_refit_pred.csv")
    pd.DataFrame({"ID": test_df["ID"].values, "clicked": pred}).to_csv(refit_file, index=False)
    print("💾 Refit pred saved:", refit_file)

    seed_test_preds_refit[seed] = pred
    del model, train_df, test_df
    gc.collect()



🧠 Refit seed=42 | refit_iters=1482 | mode=FULL


/tmp/ipykernel_449002/1363397661.py:42: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()


💾 Refit pred saved: ./outputs/CB_full_seed42_refit_pred.csv

🧠 Refit seed=106 | refit_iters=1530 | mode=FULL


/tmp/ipykernel_449002/1363397661.py:42: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()


💾 Refit pred saved: ./outputs/CB_full_seed106_refit_pred.csv

🧠 Refit seed=1031 | refit_iters=1546 | mode=FULL


/tmp/ipykernel_449002/1363397661.py:42: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  auto_cat_cols = train_df[feature_cols_base].select_dtypes(include=["object","category"]).columns.tolist()


💾 Refit pred saved: ./outputs/CB_full_seed1031_refit_pred.csv


## [4] SoftVoting 앙상블 + Test 기반 평가(옵션)

In [16]:

# SoftVoting
# prefer refit preds if available
pred_source = seed_test_preds_refit if ('seed_test_preds_refit' in globals() and len(seed_test_preds_refit)>0) else seed_test_preds
seeds = list(pred_source.keys())
if len(seeds) == 0:
    raise RuntimeError("No seed predictions found. Run [3] first.")

if CFG.SEED_WEIGHTS is None:
    w = np.ones(len(seeds), dtype=float)
else:
    w = np.array([float(CFG.SEED_WEIGHTS.get(s, 1.0)) for s in seeds], dtype=float)
w = w / w.sum()

pred_mat = np.vstack([pred_source[s] for s in seeds])  # (n_seeds, n_test)
ens_pred = (pred_mat.T @ w).astype(float)

ens_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_{mode}_softvoting_pred.csv")
pd.DataFrame({"ID": test_base["ID"].values, "clicked": ens_pred}).to_csv(ens_file, index=False)
print("💾 SoftVoting saved:", ens_file)

# Optional test evaluation
def load_test_labels():
    if "clicked" in test_base.columns:
        return test_base[["ID","clicked"]].copy()
    if CFG.TEST_LABEL_FILE is None:
        return None
    path = CFG.TEST_LABEL_FILE
    if not os.path.exists(path):
        raise FileNotFoundError(f"TEST_LABEL_FILE not found: {path}")
    if path.endswith(".parquet"):
        lab = pd.read_parquet(path)
    else:
        lab = pd.read_csv(path)
    if "ID" not in lab.columns or "clicked" not in lab.columns:
        raise KeyError("TEST_LABEL_FILE must include columns: ID, clicked")
    return lab[["ID","clicked"]].copy()

labels = load_test_labels()
if labels is None:
    print("ℹ️ Test labels not provided -> skip test evaluation.")
else:
    merged = pd.DataFrame({"ID": test_base["ID"].values, "pred": ens_pred}).merge(labels, on="ID", how="inner")
    y_true = merged["clicked"].values.astype(int)
    y_pred = np.clip(merged["pred"].values.astype(float), EPS, 1-EPS)
    ap = float(average_precision_score(y_true, y_pred))
    ll = float(log_loss(y_true, y_pred))
    ts = float(toss_score(y_true, y_pred))
    print(f"✅ [TEST EVAL] AP={ap:.6f} | LogLoss={ll:.6f} | Toss={ts:.6f} | n={len(merged)}")


💾 SoftVoting saved: ./outputs/CB_full_softvoting_pred.csv
ℹ️ Test labels not provided -> skip test evaluation.
